In [ ]:
%matplotlib widget


# Constrained Layout Guide

How to use constrained-layout to fit plots within your figure cleanly.

*constrained_layout* automatically adjusts subplots and decorations like
legends and colorbars so that they fit in the figure window while still
preserving, as best they can, the logical layout requested by the user.

*constrained_layout* is similar to `tight_layout`,
but uses a constraint solver to determine the size of axes that allows
them to fit.

*constrained_layout* typically needs to be activated before any axes are
added to a figure. Two ways of doing so are

* using the respective argument to `pyplot.subplots` or
  `pyplot.figure`, e.g.:

```python
      plt.subplots(layout="constrained")
```

* activate it via `rcParams`,
  like:
  
```python
      plt.rcParams['figure.constrained_layout.use'] = True
```

Those are described in detail throughout the following sections.

## Simple Example

In Matplotlib, the location of axes (including subplots) are specified in
normalized figure coordinates. It can happen that your axis labels or
titles (or sometimes even ticklabels) go outside the figure area, and are thus
clipped.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import numpy as np

plt.rcParams['savefig.facecolor'] = "0.8"
plt.rcParams['figure.figsize'] = 4.5, 4.
plt.rcParams['figure.max_open_warning'] = 50


def example_plot(ax, fontsize=12, hide_labels=False):
    ax.plot([1, 2])

    ax.locator_params(nbins=3)
    if hide_labels:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
    else:
        ax.set_xlabel('x-label', fontsize=fontsize)
        ax.set_ylabel('y-label', fontsize=fontsize)
        ax.set_title('Title', fontsize=fontsize)

fig, ax = plt.subplots(layout=None)
example_plot(ax, fontsize=24)

To prevent this, the location of axes needs to be adjusted. For
subplots, this can be done manually by adjusting the subplot parameters
using `Figure.subplots_adjust`. However, specifying your figure with the
``layout="constrained"`` keyword argument will do the adjusting
automatically.



In [ ]:
fig, ax = plt.subplots(layout="constrained")
example_plot(ax, fontsize=24)

When you have multiple subplots, often you see labels of different
axes overlapping each other.



In [ ]:
fig, axs = plt.subplots(2, 2, layout=None)
for ax in axs.flat:
    example_plot(ax)

Specifying ``layout="constrained"`` in the call to ``plt.subplots``
causes the layout to be properly constrained.



In [ ]:
fig, axs = plt.subplots(2, 2, layout="constrained")
for ax in axs.flat:
    example_plot(ax)

## Colorbars

If you create a colorbar with `Figure.colorbar`,
you need to make room for it.  `constrained_layout` does this
automatically.  Note that if you specify `use_gridspec=True` it will be
ignored because this option is made for improving the layout via
`tight_layout`.

<div class="alert alert-info"><h4>Note</h4><p>For the `Axes.pcolormesh` keyword arguments (`pc_kwargs`) we use a
  dictionary. Below we will assign one colorbar to a number of axes each
  containing a `ScalarMappable`; specifying the norm and colormap
  ensures the colorbar is accurate for all the axes.</p></div>



In [ ]:
arr = np.arange(100).reshape((10, 10))
norm = mcolors.Normalize(vmin=0., vmax=100.)
# see note above: this makes all pcolormesh calls consistent:
pc_kwargs = {'rasterized': True, 'cmap': 'viridis', 'norm': norm}
fig, ax = plt.subplots(figsize=(4, 4), layout="constrained")
im = ax.pcolormesh(arr, **pc_kwargs)
fig.colorbar(im, ax=ax, shrink=0.6)

If you specify a list of axes (or other iterable container) to the
``ax`` argument of ``colorbar``, constrained_layout will take space from
the specified axes.



In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(4, 4), layout="constrained")
for ax in axs.flat:
    im = ax.pcolormesh(arr, **pc_kwargs)
fig.colorbar(im, ax=axs, shrink=0.6)

If you specify a list of axes from inside a grid of axes, the colorbar
will steal space appropriately, and leave a gap, but all subplots will
still be the same size.



In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(4, 4), layout="constrained")
for ax in axs.flat:
    im = ax.pcolormesh(arr, **pc_kwargs)
fig.colorbar(im, ax=axs[1:, ][:, 1], shrink=0.8)
fig.colorbar(im, ax=axs[:, -1], shrink=0.6)

## Suptitle

``constrained_layout`` can also make room for `Figure.suptitle`.



In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(4, 4), layout="constrained")
for ax in axs.flat:
    im = ax.pcolormesh(arr, **pc_kwargs)
fig.colorbar(im, ax=axs, shrink=0.6)
fig.suptitle('Big Suptitle')

## Padding and Spacing

Padding between axes is controlled in the horizontal by *w_pad* and
*wspace*, and vertical by *h_pad* and *hspace*.  These can be edited
via `~.layout_engine.ConstrainedLayoutEngine.set`.  *w/h_pad* are
the minimum space around the axes in units of inches:



In [ ]:
fig, axs = plt.subplots(2, 2, layout="constrained")
for ax in axs.flat:
    example_plot(ax, hide_labels=True)
fig.get_layout_engine().set(w_pad=4 / 72, h_pad=4 / 72, hspace=0,
                            wspace=0)

Spacing between subplots is further set by *wspace* and *hspace*. These
are specified as a fraction of the size of the subplot group as a whole.
If these values are smaller than *w_pad* or *h_pad*, then the fixed pads are
used instead. Note in the below how the space at the edges doesn't change
from the above, but the space between subplots does.



In [ ]:
fig, axs = plt.subplots(2, 2, layout="constrained")
for ax in axs.flat:
    example_plot(ax, hide_labels=True)
fig.get_layout_engine().set(w_pad=4 / 72, h_pad=4 / 72, hspace=0.2,
                            wspace=0.2)

If there are more than two columns, the *wspace* is shared between them,
so here the wspace is divided in two, with a *wspace* of 0.1 between each
column:



In [ ]:
fig, axs = plt.subplots(2, 3, layout="constrained")
for ax in axs.flat:
    example_plot(ax, hide_labels=True)
fig.get_layout_engine().set(w_pad=4 / 72, h_pad=4 / 72, hspace=0.2,
                            wspace=0.2)

GridSpecs also have optional *hspace* and *wspace* keyword arguments,
that will be used instead of the pads set by ``constrained_layout``:



In [ ]:
fig, axs = plt.subplots(2, 2, layout="constrained",
                        gridspec_kw={'wspace': 0.3, 'hspace': 0.2})
for ax in axs.flat:
    example_plot(ax, hide_labels=True)
# this has no effect because the space set in the gridspec trumps the
# space set in constrained_layout.
fig.get_layout_engine().set(w_pad=4 / 72, h_pad=4 / 72, hspace=0.0,
                            wspace=0.0)

### Spacing with colorbars

Colorbars are placed a distance *pad* from their parent, where *pad*
is a fraction of the width of the parent(s).  The spacing to the
next subplot is then given by *w/hspace*.



In [ ]:
fig, axs = plt.subplots(2, 2, layout="constrained")
pads = [0, 0.05, 0.1, 0.2]
for pad, ax in zip(pads, axs.flat):
    pc = ax.pcolormesh(arr, **pc_kwargs)
    fig.colorbar(pc, ax=ax, shrink=0.6, pad=pad)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title(f'pad: {pad}')
fig.get_layout_engine().set(w_pad=2 / 72, h_pad=2 / 72, hspace=0.2,
                            wspace=0.2)


## Grids of fixed aspect-ratio Axes: "compressed" layout

``constrained_layout`` operates on the grid of "original" positions for
axes. However, when Axes have fixed aspect ratios, one side is usually made
shorter, and leaves large gaps in the shortened direction. In the following,
the Axes are square, but the figure quite wide so there is a horizontal gap:



In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(5, 3),
                        sharex=True, sharey=True, layout="constrained")
for ax in axs.flat:
    ax.imshow(arr)
fig.suptitle("fixed-aspect plots, layout='constrained'")

One obvious way of fixing this is to make the figure size more square,
however, closing the gaps exactly requires trial and error.  For simple grids
of Axes we can use ``layout="compressed"`` to do the job for us:



In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(5, 3),
                        sharex=True, sharey=True, layout='compressed')
for ax in axs.flat:
    ax.imshow(arr)
fig.suptitle("fixed-aspect plots, layout='compressed'")

## Manually turning off ``constrained_layout``

``constrained_layout`` usually adjusts the axes positions on each draw
of the figure.  If you want to get the spacing provided by
``constrained_layout`` but not have it update, then do the initial
draw and then call ``fig.set_layout_engine(None)``.
This is potentially useful for animations where the tick labels may
change length.

Note that ``constrained_layout`` is turned off for ``ZOOM`` and ``PAN``
GUI events for the backends that use the toolbar.  This prevents the
axes from changing position during zooming and panning.


## Limitations

### Incompatible functions

``constrained_layout`` will work with `.pyplot.subplot`, but only if the
number of rows and columns is the same for each call.
The reason is that each call to `.pyplot.subplot` will create a new
`.GridSpec` instance if the geometry is not the same, and
``constrained_layout``.  So the following works fine:



In [ ]:
fig = plt.figure(layout="constrained")

ax1 = plt.subplot(2, 2, 1)
ax2 = plt.subplot(2, 2, 3)
# third axes that spans both rows in second column:
ax3 = plt.subplot(2, 2, (2, 4))

example_plot(ax1)
example_plot(ax2)
example_plot(ax3)
plt.suptitle('Homogenous nrows, ncols')

but the following leads to a poor layout:



In [ ]:
fig = plt.figure(layout="constrained")

ax1 = plt.subplot(2, 2, 1)
ax2 = plt.subplot(2, 2, 3)
ax3 = plt.subplot(1, 2, 2)

example_plot(ax1)
example_plot(ax2)
example_plot(ax3)
plt.suptitle('Mixed nrows, ncols')

Similarly,
`~matplotlib.pyplot.subplot2grid` works with the same limitation
that nrows and ncols cannot change for the layout to look good.



In [ ]:
fig = plt.figure(layout="constrained")

ax1 = plt.subplot2grid((3, 3), (0, 0))
ax2 = plt.subplot2grid((3, 3), (0, 1), colspan=2)
ax3 = plt.subplot2grid((3, 3), (1, 0), colspan=2, rowspan=2)
ax4 = plt.subplot2grid((3, 3), (1, 2), rowspan=2)

example_plot(ax1)
example_plot(ax2)
example_plot(ax3)
example_plot(ax4)
fig.suptitle('subplot2grid')

### Other Caveats

* `constrained_layout` only considers ticklabels, axis labels, titles, and
  legends.  Thus, other artists may be clipped and also may overlap.

* It assumes that the extra space needed for ticklabels, axis labels,
  and titles is independent of original location of axes. This is
  often true, but there are rare cases where it is not.

* There are small differences in how the backends handle rendering fonts,
  so the results will not be pixel-identical.

* An artist using axes coordinates that extend beyond the axes
  boundary will result in unusual layouts when added to an
  axes. This can be avoided by adding the artist directly to the
  `Figure` using
  `Figure.add_artist`. See
  `ConnectionPatch` for an example.




# Tight Layout guide

How to use tight-layout to fit plots within your figure cleanly.

*tight_layout* automatically adjusts subplot params so that the
subplot(s) fits in to the figure area. This is an experimental
feature and may not work for some cases. It only checks the extents
of ticklabels, axis labels, and titles.


## Simple Example

In matplotlib, the location of axes (including subplots) are specified in
normalized figure coordinates. It can happen that your axis labels or
titles (or sometimes even ticklabels) go outside the figure area, and are thus
clipped.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['savefig.facecolor'] = "0.8"


def example_plot(ax, fontsize=12):
    ax.plot([1, 2])

    ax.locator_params(nbins=3)
    ax.set_xlabel('x-label', fontsize=fontsize)
    ax.set_ylabel('y-label', fontsize=fontsize)
    ax.set_title('Title', fontsize=fontsize)

plt.close('all')
fig, ax = plt.subplots()
example_plot(ax, fontsize=24)

To prevent this, the location of axes needs to be adjusted. For
subplots, this can be done manually by adjusting the subplot parameters
using `Figure.subplots_adjust`. `Figure.tight_layout` does this
automatically.



In [ ]:
fig, ax = plt.subplots()
example_plot(ax, fontsize=24)
plt.tight_layout()

Note that `pyplot.tight_layout` will only adjust the
subplot params when it is called.  In order to perform this adjustment each
time the figure is redrawn, you can call `fig.set_tight_layout(True)`, or,
equivalently, set the `figure.autolayout` rcParam to `True`.

When you have multiple subplots, often you see labels of different
axes overlapping each other.



In [ ]:
plt.close('all')

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
example_plot(ax1)
example_plot(ax2)
example_plot(ax3)
example_plot(ax4)

`pyplot.tight_layout` will also adjust spacing between
subplots to minimize the overlaps.



In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
example_plot(ax1)
example_plot(ax2)
example_plot(ax3)
example_plot(ax4)
plt.tight_layout()

`pyplot.tight_layout` can take keyword arguments of
*pad*, *w_pad* and *h_pad*. These control the extra padding around the
figure border and between subplots. The pads are specified in fraction
of fontsize.



In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
example_plot(ax1)
example_plot(ax2)
example_plot(ax3)
example_plot(ax4)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

`pyplot.tight_layout` will work even if the sizes of
subplots are different as far as their grid specification is
compatible. In the example below, *ax1* and *ax2* are subplots of a 2x2
grid, while *ax3* is of a 1x2 grid.



In [ ]:
plt.close('all')
fig = plt.figure()

ax1 = plt.subplot(221)
ax2 = plt.subplot(223)
ax3 = plt.subplot(122)

example_plot(ax1)
example_plot(ax2)
example_plot(ax3)

plt.tight_layout()

It works with subplots created with
`pyplot.subplot2grid`. In general, subplots created
from the gridspec will work.



In [ ]:
plt.close('all')
fig = plt.figure()

ax1 = plt.subplot2grid((3, 3), (0, 0))
ax2 = plt.subplot2grid((3, 3), (0, 1), colspan=2)
ax3 = plt.subplot2grid((3, 3), (1, 0), colspan=2, rowspan=2)
ax4 = plt.subplot2grid((3, 3), (1, 2), rowspan=2)

example_plot(ax1)
example_plot(ax2)
example_plot(ax3)
example_plot(ax4)

plt.tight_layout()

Although not thoroughly tested, it seems to work for subplots with
aspect != "auto" (e.g., axes with images).



In [ ]:
arr = np.arange(100).reshape((10, 10))

plt.close('all')
fig = plt.figure(figsize=(5, 4))

ax = plt.subplot()
im = ax.imshow(arr, interpolation="none")

plt.tight_layout()

## Caveats

* `pyplot.tight_layout` considers all artists on the axes by
  default.  To remove an artist from the layout calculation you can call
  `Artist.set_in_layout`.

* `tight_layout` assumes that the extra space needed for artists is
  independent of the original location of axes. This is often true, but there
  are rare cases where it is not.

* `pad=0` can clip some texts by a few pixels. This may be a bug or
  a limitation of the current algorithm, and it is not clear why it
  happens. Meanwhile, use of pad larger than 0.3 is recommended.

### Resources

- https://matplotlib.org/stable/tutorials/intermediate/constrainedlayout_guide.html
- https://matplotlib.org/stable/tutorials/intermediate/tight_layout_guide.html
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/demo_constrained_layout.html
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/demo_tight_layout.html